In [20]:
class StateException(Exception):
    pass

class State:
    def __init__(self, state_store, state, cm_raises_exception=None, *args, **kwargs):
        self.state = state
        self.state_store = state_store
        self.state_store.update(self.state)
    
    def __repr__(self):
        return f"<self.__class__.__name__ {self.state}>"

class ContextManagerState(State):
    def __init__(self, state_store, state, cm_raises_exception=None, *args, **kwargs):
        self.cm_raises_exception = cm_raises_exception

        if cm_raises_exception:
            self.restore_state = None
        else:
            # restore state for when used as context manager.
            self.restore_state = dict(state_store.state)
        
        print("--- ", state)
        print("..r", self.restore_state)
        State.__init__(self, state_store, state)
    
    def __enter__(self):
        """
        Context manager.
        
        Throw exception if self.cm_raises_exception is set.
        
        Set state back to restore state, then push new color onto stack.
        """
        if self.cm_raises_exception:
            ex = self.cm_raises_exception[0]
            
            if len(self.cm_raises_exception) == 1:
                raise ex()
                
            args = self.cm_raises_exception[1]
            if len(self.cm_raises_exception) == 2:
                raise ex(*args)
                
            kwargs = self.cm_raises_exception[2]
            raise ex(*args, **kwargs)
            
        if self.restore_state == self.state:
            raise StateException("Context manager was already restored.")
        
        # Avoid expensive list insert, by resetting state + pushing. 
        self.state_store.update(self.restore_state)
        self.state_store.push(self.state)
        
    def __exit__(self, *args):
        self.state_store.pop()
        self.state = self.restore_state


class ColorState(ContextManagerState):
    def __init__(self, state_store, mode, value, cm_raises_exception=None):
        """
        :param state_store: Backend to store color data.
        :param mode: rgba, indexed
        :param value: channel data
        
        :param cm_raises_exception: None if use as a context manager is allowed.
        :param cm_raises_exception: (exception, args, kwargs) 
        exception to raise on use as context manager.
        """
        ContextManagerState.__init__(self, state_store, {"mode": mode, "value": value}, cm_raises_exception)

class PathState(State):
    def __init__(self, state_store, fill, stroke):
        State.__init__(self, state_store,
                       {"fill": fill, "stroke": stroke,
                       "elements": []},
                       cm_raises_exception)
        
    @property
    def elements(self):
        return self.state["elements"]


class StateStore:
    def __init__(self, state):
        self.stack = []
        self.stack.append(state)
            
    def update(self, state):
        self.state.update(state)
        return len(self.stack) -1
    
    @property
    def state(self):
        return self.stack[-1]
    
    def pop(self):
        del self.stack[-1]

    def push(self, state):
        self.stack.append(state)
        return len(self.stack) -1
    
    def __repr__(self):
        return f"<StateStack {self.stack}>"
        
# default_state = dict(
#     fill={"mode": "rgba", "value": (1., 1., 1., 1.)},
#     stroke={"mode": "rgba", "value": (0., 0., 0., 1.)},
# )
# state_store = StateStore(default_state)
# state_store

class Context:
    def __init__(self):
        default_state = dict(
            fill={"mode": "rgba", "value": (1., 1., 1., 1.)},
            stroke={"mode": "rgba", "value": (0., 0., 0., 1.)},
        )
        self.state_store = StateStore(default_state)

    def fill(self, *rgba):
        if not rgba:
            return ColorState(self.state_store, **self.state_store.state["fill"],
                             cm_raises_exception=(ValueError, ["Not allowed here."]))
        color = ColorState(self.state_store, "rgba", rgba)
        return color
    
    def __repr__(self):
        return f"<Context {self.state_store}>"
    
ctx = Context()
#print(ctx)

color = ctx.fill(1., 0., 1., 1.)
print(ctx.state_store.stack[-1])

# ctx.fill()

# with ctx.fill():
#     print("yay")

---  {'mode': 'rgba', 'value': (1.0, 0.0, 1.0, 1.0)}
..r {'fill': {'mode': 'rgba', 'value': (1.0, 1.0, 1.0, 1.0)}, 'stroke': {'mode': 'rgba', 'value': (0.0, 0.0, 0.0, 1.0)}}
{'fill': {'mode': 'rgba', 'value': (1.0, 1.0, 1.0, 1.0)}, 'stroke': {'mode': 'rgba', 'value': (0.0, 0.0, 0.0, 1.0)}, 'mode': 'rgba', 'value': (1.0, 0.0, 1.0, 1.0)}


In [ ]:
# Need nested state

#fill=ColorState

state={"fill": {"mode": "rgba", "value", [1,.2,.1, 1]}}